In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from itertools import product
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from tensorflow.keras.optimizers import Adam



In [3]:
# Loading the dataset
file_path = '/content/Alphabets_data.csv'
alphabets_data = pd.read_csv(file_path)

In [4]:
############# Data Exploration and Preprocessing #################
data_info = {
    "Number of samples": alphabets_data.shape[0],
    "Number of features": alphabets_data.shape[1],
    "Column names": alphabets_data.columns.tolist(),
    "Missing values": alphabets_data.isnull().sum().to_dict(),
    "Sample data": alphabets_data.head().to_dict()
}
data_info


{'Number of samples': 20000,
 'Number of features': 17,
 'Column names': ['letter',
  'xbox',
  'ybox',
  'width',
  'height',
  'onpix',
  'xbar',
  'ybar',
  'x2bar',
  'y2bar',
  'xybar',
  'x2ybar',
  'xy2bar',
  'xedge',
  'xedgey',
  'yedge',
  'yedgex'],
 'Missing values': {'letter': 0,
  'xbox': 0,
  'ybox': 0,
  'width': 0,
  'height': 0,
  'onpix': 0,
  'xbar': 0,
  'ybar': 0,
  'x2bar': 0,
  'y2bar': 0,
  'xybar': 0,
  'x2ybar': 0,
  'xy2bar': 0,
  'xedge': 0,
  'xedgey': 0,
  'yedge': 0,
  'yedgex': 0},
 'Sample data': {'letter': {0: 'T', 1: 'I', 2: 'D', 3: 'N', 4: 'G'},
  'xbox': {0: 2, 1: 5, 2: 4, 3: 7, 4: 2},
  'ybox': {0: 8, 1: 12, 2: 11, 3: 11, 4: 1},
  'width': {0: 3, 1: 3, 2: 6, 3: 6, 4: 3},
  'height': {0: 5, 1: 7, 2: 8, 3: 6, 4: 1},
  'onpix': {0: 1, 1: 2, 2: 6, 3: 3, 4: 1},
  'xbar': {0: 8, 1: 10, 2: 10, 3: 5, 4: 8},
  'ybar': {0: 13, 1: 5, 2: 6, 3: 9, 4: 6},
  'x2bar': {0: 0, 1: 5, 2: 2, 3: 4, 4: 6},
  'y2bar': {0: 6, 1: 4, 2: 6, 3: 6, 4: 6},
  'xybar': {0: 6, 1:

In [6]:
# Separating features and target variable
features = alphabets_data.drop('letter', axis=1)
target = alphabets_data['letter']

In [7]:
# Normalizing features using MinMaxScaler
scaler = MinMaxScaler()
normalized_features = pd.DataFrame(scaler.fit_transform(features), columns=features.columns)

In [8]:
# Summary of normalized features
normalized_summary = normalized_features.describe()
normalized_summary

,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgey,yedge,yedgex
count,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000
mean,0.268237,0.469033,0.341457,0.358163,0.233723,0.459840,0.500030,0.308573,0.345243,0.552137,0.430267,0.528600,0.203073,0.555923,0.246117,0.520080
std,0.127547,0.220304,0.134305,0.150759,0.146031,0.135069,0.155024,0.179998,0.158722,0.165898,0.175405,0.138708,0.155503,0.103115,0.171138,0.107831
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.200000,0.333333,0.266667,0.266667,0.133333,0.400000,0.400000,0.200000,0.266667,0.466667,0.333333,0.466667,0.066667,0.533333,0.133333,0.466667
50%,0.266667,0.466667,0.333333,0.400000,0.200000,0.466667,0.466667,0.266667,0.333333,0.533333,0.400000,0.533333,0.200000,0.533333,0.200000,0.533333
75%,0.333333,0.600000,0.400000,0.466667,0.333333,0.533333,0.600000,0.400000,0.466667,0.666667,0.533333,0.600000,0.266667,0.600000,0.333333,0.600000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
############### Model Implementation ############################
# Encoding the target variable
label_encoder = LabelEncoder()
encoded_target = label_encoder.fit_transform(target)

In [10]:
# Splitting the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    normalized_features, encoded_target, test_size=0.2, random_state=42
)


In [11]:
# Construct the ANN model
model = Sequential([
    Input(shape=(X_train.shape[1],)),  # Explicit input layer
    Dense(64, activation='relu'),     # Hidden layer
    Dense(26, activation='softmax')   # Output layer for 26 classes
])

In [12]:
# Compile the model
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [13]:
# Training the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=1)


Epoch 1/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.1291 - loss: 3.1546 - val_accuracy: 0.3734 - val_loss: 2.6287
Epoch 2/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4061 - loss: 2.4516 - val_accuracy: 0.5147 - val_loss: 2.0495
Epoch 3/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5166 - loss: 1.9401 - val_accuracy: 0.5784 - val_loss: 1.7483
Epoch 4/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5801 - loss: 1.6791 - val_accuracy: 0.6025 - val_loss: 1.5721
Epoch 5/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6055 - loss: 1.5219 - val_accuracy: 0.6353 - val_loss: 1.4541
Epoch 6/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6411 - loss: 1.3889 - val_accuracy: 0.6584 - val_loss: 1.3681
Epoch 7/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6640 - loss: 1.3012 - val_accuracy: 0.6597 - val_loss: 1.3078
Epoch 8/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6738 - loss: 1.2604 - val_accuracy: 0.

In [14]:
# Making predictions on the test set
test_predictions = model.predict(X_test)


125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [15]:
# Converting predictions to class labels
predicted_classes = np.argmax(test_predictions, axis=1)


In [17]:
# Model summary and accuracy

test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")


Test Loss: 1.139926791191101, Test Accuracy: 0.702750027179718


# Hyperparameter Tuning #########################

In [19]:
# Encode target variable
label_encoder = LabelEncoder()
encoded_target = label_encoder.fit_transform(target)


In [20]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    normalized_features, encoded_target, test_size=0.2, random_state=42
)

In [21]:
# Define hyperparameter space
hidden_layers = [1, 2, 3]
neurons_per_layer = [32, 64, 128]
activations = ['relu', 'tanh']
learning_rates = [0.001, 0.01, 0.1]
batch_sizes = [16, 32]


In [22]:
# Grid search setup
results = []
for layers, neurons, activation, lr, batch_size in product(
    hidden_layers, neurons_per_layer, activations, learning_rates, batch_sizes
):
    # Create model with current hyperparameters
    tuned_model = Sequential()
    tuned_model.add(Dense(neurons, input_dim=X_train.shape[1], activation=activation))
    for _ in range(layers - 1):
        tuned_model.add(Dense(neurons, activation=activation))
    tuned_model.add(Dense(26, activation='softmax'))  # Output layer

    # Compile the tuned model
    tuned_model.compile(
      loss='sparse_categorical_crossentropy',
      optimizer=Adam(learning_rate=lr),
      metrics=['accuracy'],
    )

    # Train model
    history = model.fit(
        X_train,
        y_train,
        epochs=10,
        batch_size=batch_size,
        validation_split=0.2,
        verbose=0,
    )

    # Evaluate on test set
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")
    results.append(
        {
            "hidden_layers": layers,
            "neurons_per_layer": neurons,
            "activation": activation,
            "learning_rate": lr,
            "batch_size": batch_size,
            "test_accuracy": test_accuracy,
        }
    )

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Test Loss: 0.8922140598297119, Test Accuracy: 0.7674999833106995


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Test Loss: 0.8243882656097412, Test Accuracy: 0.7900000214576721


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

Test Loss: 0.27121224999427795, Test Accuracy: 0.9175000190734863


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Test Loss: 0.2734995484352112, Test Accuracy: 0.9185000061988831


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Test Loss: 0.2688247561454773, Test Accuracy: 0.9167500138282776


In [23]:
# Display results
results = sorted(results, key=lambda x: x["test_accuracy"], reverse=True)
for result in results[:5]:  # Display top 5 configurations
    print(result)

{'hidden_layers': 3, 'neurons_per_layer': 128, 'activation': 'tanh', 'learning_rate': 0.001, 'batch_size': 32, 'test_accuracy': 0.9197499752044678}
{'hidden_layers': 3, 'neurons_per_layer': 128, 'activation': 'tanh', 'learning_rate': 0.001, 'batch_size': 16, 'test_accuracy': 0.918749988079071}
{'hidden_layers': 3, 'neurons_per_layer': 128, 'activation': 'tanh', 'learning_rate': 0.1, 'batch_size': 16, 'test_accuracy': 0.9185000061988831}
{'hidden_layers': 3, 'neurons_per_layer': 128, 'activation': 'relu', 'learning_rate': 0.01, 'batch_size': 32, 'test_accuracy': 0.9182500243186951}
{'hidden_layers': 3, 'neurons_per_layer': 64, 'activation': 'tanh', 'learning_rate': 0.01, 'batch_size': 32, 'test_accuracy': 0.9179999828338623}


# Model Evaluation ####################

In [24]:
# Evaluate default model performance
default_predictions = model.predict(X_test)
default_predicted_classes = np.argmax(default_predictions, axis=1)


125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [25]:
default_metrics = {
    "Accuracy": accuracy_score(y_test, default_predicted_classes),
    "Precision": precision_score(y_test, default_predicted_classes, average='weighted', zero_division=0), # Add zero_division
    "Recall": recall_score(y_test, default_predicted_classes, average='weighted', zero_division=0), # Add zero_division
    "F1-Score": f1_score(y_test, default_predicted_classes, average='weighted', zero_division=0), # Add zero_division
}


In [26]:
# Assume tuned model is already trained
tuned_predictions = tuned_model.predict(X_test)
tuned_predicted_classes = np.argmax(tuned_predictions, axis=1)


125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [28]:
# Calculate metrics for the tuned model

precision = precision_score(y_test, predicted_classes, average='weighted', zero_division=1)
recall = recall_score(y_test, predicted_classes, average='weighted', zero_division=1)
f1 = f1_score(y_test, predicted_classes, average='weighted', zero_division=1)



In [29]:
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")

Precision: 0.705434871180533
Recall: 0.70275
F1-Score: 0.6986280862438858
